In [ ]:
import numpy as np
import itertools as itr

import os as os
import sys as sys 
import pandas as pd
import warnings
import time as time
import random
from __future__ import print_function # so print doesn't show brackets

sys.path.append(os.path.join("..","Libraries","QML_lib"))
import Evo as evo
import DataBase 
import QMD
import QML
import ModelGeneration
import BayesF
import matplotlib.pyplot as plt

global paulis_list
paulis_list = {'i' : np.eye(2), 'x' : evo.sigmax(), 'y' : evo.sigmay(), 'z' : evo.sigmaz()}

warnings.filterwarnings("ignore", message='Negative weights occured', category=RuntimeWarning)



class NewQMDClass():
    #TODO: rename as ModelsDevelopmentClass when finished
    def __init__(self,
                 initial_op_list,
                 true_operator='x',
                 maxmodnum=3, 
                 max_num_qubits=4, 
                 gaussian=True,
                 max_num_layers = 10,
                 max_num_branches = 20, 
                 use_exp_custom = True
                ):
#    def __init__(self, initial_op_list, true_op_list, true_param_list):
        
        trueOp = DataBase.operator(true_operator)
        self.TrueOpName = true_operator
        self.TrueOpDim = trueOp.num_qubits
        self.InitialOpList = initial_op_list
        self.TrueOpList = trueOp.constituents_operators
#        self.TrueParamsList = [random.random() for i in self.TrueOpList] # TODO: actual true params?
        # todo set true parmams properly
        self.TrueParamsList = [0.75 for i in self.TrueOpList] # TODO: actual true params?
        #self.TrueHam = evo.getH(self.TrueParamsList, self.TrueOpList)
        self.TrueHam = np.tensordot(self.TrueParamsList, self.TrueOpList, axes=1)
        self.MaxModNum = maxmodnum #TODO: necessary?
        self.gaussian = gaussian
        self.NumModels = len(initial_op_list)
        self.NumParticles = 1000
        self.MaxQubitNumber = max_num_qubits
        self.HighestQubitNumber = int(0)
        self.MaxBranchID = max_num_branches
        self.HighestBranchID = 0
        self.MaxLayerNumber = max_num_layers
        self.BranchChampions = {}
        self.LayerChampions = {}
        self.BayesFactorsByBranch ={}
        self.BranchRankings = {}
        self.BranchBayesComputed = {}
        self.InterBranchChampions = {}
        self.GlobalEpoch = 0 
        self.UseExpCustom = use_exp_custom
        
#        for i in range(self.MaxBranchID+1):
#            self.BranchChampions[i] = 0
#        for i in range(self.MaxLayerNumber+1):
#            self.LayerChampions[i] = 0
        
        print("\nQMD has true operator: ", true_operator)
        print("QMD has true params : ", self.TrueParamsList, "\n")

        # Initialise database and lists.
        self.initiateDB()
        
    def initiateDB(self):
        
        ## TODO: Models should be initialised with appropriate TrueOp dimension -- see getListTrueOpByDimension function
        self.db, self.legacy_db, self.model_lists = \
            DataBase.launch_db(
                true_op_name = self.TrueOpName,
                gen_list = self.InitialOpList,
                true_ops = self.TrueOpList,
                true_params = self.TrueParamsList,
                redimensionalise = True,
                use_exp_custom = self.UseExpCustom,
            )
    def addModel(self, model, treeID=0):
        self.NumModels += 1
        tryAddModel = DataBase.add_model(
            model_name = model,
            running_database = self.db,
            num_particles = self.NumParticles, 
            true_op_name = self.TrueOpName,
            model_lists = self.model_lists,
            true_ops = self.TrueOpList,
            true_params = self.TrueParamsList,
            treeID = treeID,
            use_exp_custom = self.UseExpCustom,
            modelID = self.NumModels,
            redimensionalise = True,
        )
        if tryAddModel == True: 
            if DataBase.get_num_qubits(model) > self.HighestQubitNumber:
                self.HighestQubitNumber = DataBase.get_num_qubits(model)
        else: 
            self.NumModels-=int(1)

    def newBranch(self, model_list):
        self.HighestBranchID += 1
        branchID = self.HighestBranchID
        self.BranchBayesComputed[float(branchID)] = False
        for model in model_list:
            self.addModel(model, treeID=branchID)
    
    def printState(self):
        print("Branch champions: \n", self.BranchChampions)
        print("InterBranch champions: \n", self.InterBranchChampions)
        print("Branch Rankings: \n", self.BranchRankings)
        #print("Layer Champions: \n", self.LayerChampions)
            
            
    def getModelInstance(self, name):
        try: 
            instance = DataBase.get_qml_instance(self.db, name)
            return instance
        except: 
            if name in list(self.legacy_db['<Name>']):
                print("Operator in legacy databse - retired. ")
            else: 
                print("Model not found.")
    def getOperatorInstance(self, name):
        try: 
            return DataBase.get_operator_instance(self.db, name)
        except:
            if name in list(self.legacy_db['<Name>']):
                print("Operator in legacy databse - retired. ")
            else: 
                print("Operator not found.")

    def getModelDBIndex(self, name):
        return DataBase.get_location(self.db, name)

    def getModelInstanceFromID(self, model_id):
        return DataBase.model_instance_from_id(self.db, model_id)    
    
    def killModel(self, name):
        if name not in list(self.db['<Name>']):
            print("Cannot remove ", name, "; not in ", list(self.db["<Name>"]))
        else:
            print("Killing model", name)
            # Add to legacy_db
            DataBase.move_to_legacy(self.db, self.legacy_db, name)
            model_instance = self.getModelInstance(name)
            operator_instance = self.getOperatorInstance(name)
            # Remove from self.db
            self.db = DataBase.remove_model(self.db, name)
            del model_instance
            del operator_instance
    
            #TODO: plot?

    def runIQLE(self, model, num_exp=50):
        model_exists=False
        if model in list(self.db['<Name>']):
            model_exists = True
        elif model in list(self.legacy_db['<Name>']):
            print("Model ", model, " previously considered and retired.")
        
        has_model_finished = self.pullField(name=model, field='Completed')
        
        if model_exists==True and has_model_finished==False : 
            model_instance = self.getModelInstance(model)
            print("\nRunning IQLE for model: ", model)
            model_instance.UpdateModel(num_exp)
            self.updateModelRecord(name=model, field='Completed', new_value=True)
            #model_instance.BayesOnModelsWithinTrees
        else: 
            print("Model ", model ,"does not exist")

    def runAllActiveModelsIQLE(self, num_exp):
        active_models = self.db.loc[self.db['Status']=='Active']['<Name>']

        for model in active_models:
            self.runIQLE(model=model, num_exp=num_exp)
        self.GlobalEpoch += num_exp
            
        
    def updateModelRecord(self, field, name=None, model_id=None, new_value=None, increment=None):
        DataBase.update_field(
            db=self.db, 
            name=name,
            model_id=model_id,
            field=field,
            new_value=new_value,
            increment=increment
        )
    def pullField(self, name, field):
        return DataBase.pull_field(self.db, name, field)

    def statusChangeBranch(self, branchID, new_status='Saturated'):
        self.db.loc[ self.db['TreeID']==branchID , 'Status'] = new_status

    def statusChangeModel(self, model_name, new_status='Saturated'):
        self.db.loc[ self.db['<Name>']==model_name , 'Status'] = new_status
        
    def getListTrueOpByDimension(self):
        self.TrueOpListByDim = {}
        self.TrueParamByDim = {}
        for dim in range(1, 1+self.MaxDimension):
            new_op = ModelGeneration.identity_interact(subsystem=self.TrueOpName, num_qubits=dim, return_operator=True)
            self.TrueOpListByDim[dim] = new_op.constituents_operators
        for i in range(1, self.TrueOpDim+1):
            self.TrueParamByDim[i] = self.TrueParamsList
        for i in range(self.TrueOpDim+1, self.MaxDimension):
            self.TrueParamByDim[i] = [self.TrueParamsList[0]]

    def compareModels(self, log_comparison_high=50.0, num_times_to_use = 'all', model_a_id = None, model_b_id =None, model_a = None, model_b = None, name_a=None, name_b=None):
        # Either pass in name_a and name_b OR model_a and model_b
        if model_a is None and model_b is None:
            if model_a_id is not None and model_b_id is not None: 
                model_a = self.getModelInstanceFromID(model_a_id)
                model_b = self.getModelInstanceFromID(model_b_id)
            else: # if only names were passed 
                model_a = self.getModelInstance(name_a)
                model_b = self.getModelInstance(name_b)
        if model_a ==  model_b:
            print("Same models, don't need to compare.")
            return "Same Models"

        log_comparison_low = 1/log_comparison_high
        
        if num_times_to_use == 'all':
            times_a = model_a.TrackTime
        elif len(model_a.TrackTime) < num_times_to_use:
            times_a = model_a.TrackTime
        else: 
            times_a = model_a.TrackTime[num_times_to_use:]
        
        if num_times_to_use=='all':
            times_b = model_b.TrackTime
        elif len(model_b.TrackTime) < num_times_to_use:
            times_b = model_b.TrackTime
        else: 
            times_b = model_b.TrackTime[num_times_to_use:]
        gen_a = model_a.GenSimModel
        gen_b = model_b.GenSimModel

        times = times_a + times_b
        exps_a = get_exps(model_a, gen_a, times)
        exps_b = get_exps(model_b, gen_b, times)

        log_l_a =  get_log_likelihood(model_a, gen_a, exps_a)
        log_l_b =  get_log_likelihood(model_b, gen_b, exps_b)

        bayes_factor = np.exp(log_l_a - log_l_b)

        print("bayes_factor of log likelihoods between ", model_a.Name, " and ", model_b.Name, " = ", bayes_factor)
        if bayes_factor > log_comparison_high: 
            print("Point to ", model_a.Name)
            return "a"
        elif bayes_factor < log_comparison_low: 
            print("Point to ", model_b.Name)
            return "b"
        else:
            print("No real winner")  
        # todo: Add bayes_factor with mod_id's to QML class

            
    def compareModelsWithinBranch(self, treeID):
        active_models_in_tree = DataBase.active_model_ids_by_tree_id(self.db, treeID)
        
        models_points = {}
        for model_id in active_models_in_tree:
            models_points[model_id] = 0
        
        for i in range(len(active_models_in_tree)):
            mod1 = active_models_in_tree[i]
            for j in range(i, len(active_models_in_tree)): 
                mod2 = active_models_in_tree[j]
                res = self.compareModels(model_a_id = mod1, model_b_id=mod2)
                if res == "a":
                    models_points[mod1] += 1
                elif res == "b":
                    models_points[mod2] += 1
                    # todo if more than one model has max points
                    
        max_points = max(models_points.values())
        max_points_branches = [key for key, val in models_points.items() if val==max_points]
        if len(max_points_branches) > 1: 
            # todo: recompare. Fnc: compareListOfModels (rather than branch based)
            champ_id = self.compareModelList(max_points_branches, bayes_threshold=1, models_points_dict=models_points)
        else: 
            champ_id = max(models_points, key=models_points.get)
        champ_name = DataBase.model_name_from_id(self.db, champ_id)
        
        #todo list of ranked models by branch
        
        self.BranchChampions[int(treeID)] = champ_id
        for model_id in active_models_in_tree:
            self.updateModelRecord(model_id=model_id, field='Status', new_value='Deactivated')
        self.updateModelRecord(name=DataBase.model_name_from_id(self.db, champ_id), field='Status', new_value='Active')

# sorted_keys = sorted(my_dict, key=my_dict.get, reverse=True)
        ranked_model_list = sorted_keys = sorted(models_points, key=models_points.get, reverse=True)
        print("Branch: ", treeID)
        print("List : ", ranked_model_list)
        print("Models_points dict : ", models_points)

        print("TreeID=", treeID, " has type ", type(treeID))
        #print("branch computed: ", self.BranchBayesComputed[int(float(treeID))] )
        #if self.BranchBayesComputed[float(treeID)] == False: # only update self.BranchRankings the first time branch is considered
        self.BranchRankings[treeID] = ranked_model_list
        #self.BranchBayesComputed[float(treeID)] = True
            
        print("Champion of tree ", treeID, " is ", champ_name)
        return models_points, champ_id

    
    def compareModelList(self, model_list, bayes_threshold = 50, models_points_dict=None):
        models_points = {}
        for mod in model_list:
            models_points[mod] = 0
        
        for i in range(len(model_list)):
            mod1 = model_list[i]
            for j in range(i, len(model_list)):
                mod2 = model_list[j]
                if mod1 != mod2:
                    res = self.compareModels(model_a_id=mod1, model_b_id=mod2, log_comparison_high=bayes_threshold)
                    if res == "a":
                        models_points[mod1] += 1
                        if models_points_dict is not None: 
                            models_points_dict[mod1]+=1
                    elif res == "b":
                        models_points[mod2]+=1
                        if models_points_dict is not None: 
                            models_points_dict[mod2]+=1

        max_points = max(models_points.values())
        max_points_branches = [key for key, val in models_points.items() if val==max_points]
        if len(max_points_branches) > 1: 
            # todo: recompare. Fnc: compareListOfModels (rather than branch based)
            print("No distinct champion, recomputing bayes factors between : ", max_points_branches)
            champ_id = self.compareModelList(max_points_branches, bayes_threshold=1)
        else: 
            champ_id = max(models_points, key=models_points.get)
        champ_name = DataBase.model_name_from_id(self.db, champ_id)

        
        return champ_id
    
    def interBranchChampion(self, branch_list=[], global_champion=False):
        all_branches = self.db['TreeID'].unique()
        if global_champion == True: 
            trees = all_branches
        else: 
            trees = branch_list
        
        num_branches = len(trees)
        points_by_branches = [None] * num_branches
        champions_of_branches = [None] * num_branches

        for i in range(num_branches):
            treeID = trees[i]
            if treeID not in all_branches:
                print("Tree ID : ", treeID)
                warnings.warn("Tree not in database.")
                return False
            points_by_branches[i], champions_of_branches[i] = qmd.compareModelsWithinBranch(treeID)

        branch_champions_points = {}
        for c in champions_of_branches: 
            branch_champions_points[c] = 0

        for i in range(num_branches):
            mod1 = champions_of_branches[i]
            for j in range(i, num_branches):
                mod2 = champions_of_branches[j]
                if mod1!=mod2:
                    res = self.compareModels(model_a_id=mod1, model_b_id=mod2, log_comparison_high=20.0)
                    if res == "a":
                        branch_champions_points[mod1] += 1
                    elif res == "b":
                        branch_champions_points[mod2] += 1
        self.ranked_champions = sorted(branch_champions_points, reverse=True)
        
        max_points = max(branch_champions_points.values())
        max_points_branches = [key for key, val in branch_champions_points.items() if val==max_points]
        if len(max_points_branches) > 1: 
            # todo: recompare. Fnc: compareListOfModels (rather than branch based)
            print("No distinct champion, recomputing bayes factors between : ", max_points_branches)
            champ_id = self.compareModelList(max_points_branches, bayes_threshold=1, models_points_dict=branch_champions_points)
        else: 
            champ_id = max(branch_champions_points, key=branch_champions_points.get)
        champ_name = DataBase.model_name_from_id(self.db, champ_id)
        
        branch_champ_names = [DataBase.model_name_from_id(self.db, mod_id) for mod_id in champions_of_branches]
        self.statusChangeModel(champ_name, new_status = 'Active')
        
        interBranchChampListID = len(self.InterBranchChampions)
        self.InterBranchChampions[interBranchChampListID] = [trees, champ_id]
        print("Champion of branches ", trees, " is ",  champ_name)
        return champ_name, branch_champ_names
    
    def globalChampionCalculation(self):
        trees = self.db['TreeID'].unique()
        
        num_trees = len(trees)
        self.points_by_trees = [None] * num_trees
        self.champions_of_trees = [None] * num_trees

        for i in range(num_trees):
            treeID = trees[i]
            self.points_by_trees[i], self.champions_of_trees[i] = qmd.compareModelsWithinBranch(treeID)

        self.champions_points = {}
        for c in self.champions_of_trees: 
            self.champions_points[c] = 0

        for i in range(num_trees):
            mod1 = self.champions_of_trees[i]
            for j in range(i, num_trees):
                mod2 = self.champions_of_trees[j]
                if mod1!=mod2:
                    res = self.compareModels(model_a_id=mod1, model_b_id=mod2, log_comparison_high=10.0)
                    if res == "a":
                        self.champions_points[mod1] += 1
                    elif res == "b":
                        self.champions_points[mod2]+=1
        self.ranked_champions = sorted(self.champions_points, reverse=True)
        champ_id = max(self.champions_points, key=self.champions_points.get)
        champ_name = DataBase.model_name_from_id(self.db, champ_id)
        print("Champion of Champions is",  champ_name)
        
        
    def spawn(self, 
              branch_list = None, 
              num_models_to_consider=1, 
              absolute_champion=False, 
              all_branches=False
             ):

        if all_branches or branch_list is None: 
            global_champion = True
            
        overall_champ, branch_champions = self.interBranchChampion(branch_list=branch_list, global_champion=global_champion)

        print("Overall champ : ", overall_champ)
        if absolute_champion:
            print("Only using absolute champion")
            new_models = ModelGeneration.new_model_list(model_list=[overall_champ], generator='simple_ising',options=['x', 'y'])
        else: 
            print("Using branch champions list")
            print("branch champions : ", branch_champions)
            new_models = ModelGeneration.new_model_list(model_list=branch_champions, generator='simple_ising', options=['x', 'y'])
        
        print("New models to add to new branch : ", new_models)
        qmd.newBranch(model_list=new_models) 
        
        #todo probailistically append model_list with suboptimal model in any of the branches in branch_list

    def runQMD(self, num_exp = 20, max_branches= None, max_num_qubits = None):
        if max_branches is None:
            max_branches = self.MaxBranchID

        if max_num_qubits is None:
            max_num_qubits = self.MaxQubitNumber
            
        while self.HighestQubitNumber < max_num_qubits: 
            self.runAllActiveModelsIQLE(num_exp=num_exp)
            self.spawn()
            if self.HighestBranchID > max_branches:
                break
            
        self.runAllActiveModelsIQLE(num_exp=num_exp)
        final_winner, final_branch_winners = self.interBranchChampion(global_champion=True)
        print("Final winner = ", final_winner)
        
def get_exps(model, gen, times):

    exps = np.empty(len(times), dtype=gen.expparams_dtype)
    exps['t'] = times

    for i in range(1, len(gen.expparams_dtype)):
        col_name = 'w_'+str(i)
        exps[col_name] = model.FinalParams[i-1,0] ## TODO: should be model.NewEval[i-1]???
    return exps

def get_log_likelihood(model, gen, exps):
    import copy
    updater = copy.deepcopy(model.Updater)
    data = gen.simulate_experiment(model.SimParams, exps)[0][0]
    updater.batch_update(data, exps, resample_interval=100)
    log_likelihood = np.sum(updater.log_total_likelihood)
    del updater
    return log_likelihood        

/usr/local/lib/python2.7/dist-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)
/usr/local/lib/python2.7/dist-packages/qinfer/parallel.py:52: UserWarning: Could not import IPython parallel. Parallelization support will be disabled.
  "Could not import IPython parallel. "


In [ ]:
import random
num_exp = 50

#max_dim=2
#dims = range(1,max_dim+1)
#num_qubits=random.choice(dims)

true_op = 'xTx'
generators = ['x', 'y']

qmd = NewQMDClass(initial_op_list=generators, true_operator=true_op)
# qmd.runQMD(num_exp=num_exp, max_branches=2, max_num_qubits=1)

for use_exp in [True, False]:
    print("\n\n\n\n USE EXP HAM: ", use_exp)
    for i in range(15):
        print("\n\n\n i= ", i)
        qmd = NewQMDClass(initial_op_list=generators, true_operator=true_op, use_exp_custom=use_exp)
        qmd.runIQLE(model='x', num_exp=num_exp)
        qmd.runIQLE(model='y', num_exp=num_exp)

        for j in range(100):
            print("\n\n j= ", j)
            qmd.compareModels(model_a_id=0, model_b_id=1)
#### Below: results with exp_ham turned on


QMD has true operator:  xTx
QMD has true params :  [0.75] 

Redimensionalising
Before dimensionalising name. Name =  x true_dim =  2
Model  x  not previously considered -- adding.
Initialization Ready
Redimensionalising
Before dimensionalising name. Name =  y true_dim =  2
Model  y  not previously considered -- adding.
Initialization Ready




 USE EXP HAM:  True



 i=  0

QMD has true operator:  xTx
QMD has true params :  [0.75] 

Redimensionalising
Before dimensionalising name. Name =  x true_dim =  2
Model  x  not previously considered -- adding.
Initialization Ready
Redimensionalising
Before dimensionalising name. Name =  y true_dim =  2
Model  y  not previously considered -- adding.
Initialization Ready

Running IQLE for model:  x
Initial time selected > 0.637311928609
Final time selected > 2.4764217497
Final Parameters mean and stdev:[ 0.29658912  0.28583024]

Running IQLE for model:  y
Initial time selected > 1.37544940721
Final time selected > 6.69673264852
Final Parameters m

bayes_factor of log likelihoods between  xTi  and  yTi  =  0.000105160329213
Point to  yTi


 j=  73
bayes_factor of log likelihoods between  xTi  and  yTi  =  9.84412892399e-07
Point to  yTi


 j=  74
bayes_factor of log likelihoods between  xTi  and  yTi  =  1.49623576936e-07
Point to  yTi


 j=  75
bayes_factor of log likelihoods between  xTi  and  yTi  =  5.28481200995e-06
Point to  yTi


 j=  76
bayes_factor of log likelihoods between  xTi  and  yTi  =  2.90155636914e-11
Point to  yTi


 j=  77
bayes_factor of log likelihoods between  xTi  and  yTi  =  5.79339234233e-09
Point to  yTi


 j=  78
bayes_factor of log likelihoods between  xTi  and  yTi  =  3.166280922e-08
Point to  yTi


 j=  79
bayes_factor of log likelihoods between  xTi  and  yTi  =  680.826985212
Point to  xTi


 j=  80
bayes_factor of log likelihoods between  xTi  and  yTi  =  1561.88104735
Point to  xTi


 j=  81
bayes_factor of log likelihoods between  xTi  and  yTi  =  4.29762011279e-06
Point to  yTi


 j=  82


bayes_factor of log likelihoods between  xTi  and  yTi  =  5.11503495132e-06
Point to  yTi


 j=  49
bayes_factor of log likelihoods between  xTi  and  yTi  =  2.03362092276e-05
Point to  yTi


 j=  50
bayes_factor of log likelihoods between  xTi  and  yTi  =  0.000163354589827
Point to  yTi


 j=  51
bayes_factor of log likelihoods between  xTi  and  yTi  =  6.84396151947e-06
Point to  yTi


 j=  52
bayes_factor of log likelihoods between  xTi  and  yTi  =  7.65603572119e-05
Point to  yTi


 j=  53
bayes_factor of log likelihoods between  xTi  and  yTi  =  0.00114741106169
Point to  yTi


 j=  54
bayes_factor of log likelihoods between  xTi  and  yTi  =  1.95576788251e-08
Point to  yTi


 j=  55
bayes_factor of log likelihoods between  xTi  and  yTi  =  0.00977598497483
Point to  yTi


 j=  56
bayes_factor of log likelihoods between  xTi  and  yTi  =  3.36922279597e-06
Point to  yTi


 j=  57
bayes_factor of log likelihoods between  xTi  and  yTi  =  5.11265786368e-06
Point to  yTi




bayes_factor of log likelihoods between  xTi  and  yTi  =  0.00243037258668
Point to  yTi


 j=  25
bayes_factor of log likelihoods between  xTi  and  yTi  =  2.83999659523e-06
Point to  yTi


 j=  26
bayes_factor of log likelihoods between  xTi  and  yTi  =  74.8290028815
Point to  xTi


 j=  27
bayes_factor of log likelihoods between  xTi  and  yTi  =  127.287696528
Point to  xTi


 j=  28
bayes_factor of log likelihoods between  xTi  and  yTi  =  0.890069988852
No real winner


 j=  29
bayes_factor of log likelihoods between  xTi  and  yTi  =  0.266851499749
No real winner


 j=  30
bayes_factor of log likelihoods between  xTi  and  yTi  =  1.40806922648
No real winner


 j=  31
bayes_factor of log likelihoods between  xTi  and  yTi  =  1.16813510199e-05
Point to  yTi


 j=  32
bayes_factor of log likelihoods between  xTi  and  yTi  =  12895.3916061
Point to  xTi


 j=  33
bayes_factor of log likelihoods between  xTi  and  yTi  =  35022.1940017
Point to  xTi


 j=  34
bayes_factor o

bayes_factor of log likelihoods between  xTi  and  yTi  =  0.000345615284235
Point to  yTi


 j=  2
bayes_factor of log likelihoods between  xTi  and  yTi  =  13.7017317576
No real winner


 j=  3
bayes_factor of log likelihoods between  xTi  and  yTi  =  0.0772498595395
No real winner


 j=  4
bayes_factor of log likelihoods between  xTi  and  yTi  =  60.4345371745
Point to  xTi


 j=  5
bayes_factor of log likelihoods between  xTi  and  yTi  =  4335.32314357
Point to  xTi


 j=  6
bayes_factor of log likelihoods between  xTi  and  yTi  =  2.89520155609e-07
Point to  yTi


 j=  7
bayes_factor of log likelihoods between  xTi  and  yTi  =  0.172439480799
No real winner


 j=  8
bayes_factor of log likelihoods between  xTi  and  yTi  =  0.28085510651
No real winner


 j=  9
bayes_factor of log likelihoods between  xTi  and  yTi  =  0.213706013491
No real winner


 j=  10
bayes_factor of log likelihoods between  xTi  and  yTi  =  0.00197460642809
Point to  yTi


 j=  11
bayes_factor of lo

bayes_factor of log likelihoods between  xTi  and  yTi  =  0.51992128165
No real winner


 j=  86
bayes_factor of log likelihoods between  xTi  and  yTi  =  0.0539196575465
No real winner


 j=  87
bayes_factor of log likelihoods between  xTi  and  yTi  =  5.84939248308
No real winner


 j=  88
bayes_factor of log likelihoods between  xTi  and  yTi  =  43.743977152
No real winner


 j=  89
bayes_factor of log likelihoods between  xTi  and  yTi  =  0.00530880310893
Point to  yTi


 j=  90
bayes_factor of log likelihoods between  xTi  and  yTi  =  0.000173574462602
Point to  yTi


 j=  91
bayes_factor of log likelihoods between  xTi  and  yTi  =  2.13854969604
No real winner


 j=  92
bayes_factor of log likelihoods between  xTi  and  yTi  =  472755.24472
Point to  xTi


 j=  93
bayes_factor of log likelihoods between  xTi  and  yTi  =  192912.167537
Point to  xTi


 j=  94
bayes_factor of log likelihoods between  xTi  and  yTi  =  189.83586739
Point to  xTi


 j=  95
bayes_factor of log

bayes_factor of log likelihoods between  xTi  and  yTi  =  1.53959398635e-10
Point to  yTi


 j=  63
bayes_factor of log likelihoods between  xTi  and  yTi  =  4251839998.11
Point to  xTi


 j=  64
bayes_factor of log likelihoods between  xTi  and  yTi  =  1627917.22792
Point to  xTi


 j=  65
bayes_factor of log likelihoods between  xTi  and  yTi  =  38206370841.0
Point to  xTi


 j=  66
bayes_factor of log likelihoods between  xTi  and  yTi  =  3.36324872867e+13
Point to  xTi


 j=  67
bayes_factor of log likelihoods between  xTi  and  yTi  =  332305522225.0
Point to  xTi


 j=  68
bayes_factor of log likelihoods between  xTi  and  yTi  =  3124162.83715
Point to  xTi


 j=  69
bayes_factor of log likelihoods between  xTi  and  yTi  =  98164585954.2
Point to  xTi


 j=  70
bayes_factor of log likelihoods between  xTi  and  yTi  =  0.00212757497999
Point to  yTi


 j=  71
bayes_factor of log likelihoods between  xTi  and  yTi  =  16534420.5281
Point to  xTi


 j=  72
bayes_factor of lo

bayes_factor of log likelihoods between  xTi  and  yTi  =  5.60691302363e-14
Point to  yTi


 j=  39
bayes_factor of log likelihoods between  xTi  and  yTi  =  1.33523444088e-07
Point to  yTi


 j=  40
bayes_factor of log likelihoods between  xTi  and  yTi  =  1.77326896328e-18
Point to  yTi


 j=  41
bayes_factor of log likelihoods between  xTi  and  yTi  =  9.01940571434e-18
Point to  yTi


 j=  42
bayes_factor of log likelihoods between  xTi  and  yTi  =  1.80757802299e-21
Point to  yTi


 j=  43
bayes_factor of log likelihoods between  xTi  and  yTi  =  4.57088085404e-14
Point to  yTi


 j=  44
bayes_factor of log likelihoods between  xTi  and  yTi  =  1.95001628474e-12
Point to  yTi


 j=  45
bayes_factor of log likelihoods between  xTi  and  yTi  =  8.01643115823e-07
Point to  yTi


 j=  46
bayes_factor of log likelihoods between  xTi  and  yTi  =  1.30342682257e-12
Point to  yTi


 j=  47
bayes_factor of log likelihoods between  xTi  and  yTi  =  1.62911858309e-14
Point to  yTi


In [ ]:
import random
num_exp = 100

#max_dim=2
#dims = range(1,max_dim+1)
#num_qubits=random.choice(dims)

true_op = 'xTx'
generators = ['x', 'y']

qmd = NewQMDClass(initial_op_list=generators, true_operator=true_op)
# qmd.runQMD(num_exp=num_exp, max_branches=2, max_num_qubits=1)
qmd.runIQLE(model='x', num_exp=num_exp)
qmd.runIQLE(model='y', num_exp=num_exp)

for i in range(6):
    qmd.compareModels(model_a_id=0, model_b_id=1)
#### Below: results without exp_ham turned on

In [ ]:
qmd.db

In [ ]:
def inspect_model(name):
    print("\nmodel name: ", name)
    print("experiments done ", qmd.getModelInstance(name).NumExperimentsToDate)
    print("times: ",  qmd.getModelInstance(name).TrackTime)

for mod in ['x', 'y']:
    inspect_model(mod)

In [ ]:
qmd.getModelInstance('xTxTTx').NumExperimentsToDate

## Partial trace development

In [ ]:
import hamiltonian_exponentiation as h
def evolve_state(ham, t, input_state):
    # todo : check dimensionality of input state vs hamiltonian
    unitary = h.exp_ham(ham, t)
    return unitary @ input_state